## Import packages

In [ ]:
import os
import pandas as pd 
import numpy as np
import geopandas as gpd  
import rasterio  
import matplotlib.pyplot as plt  
from rasterio.plot import show as rio_show  

In [ ]:
import sys
module_path = os.path.abspath('../')
sys.path.append(module_path)
from mobile_capacity.capacity import Capacity

In [ ]:
pd.set_option('display.max_columns', None)

## Set analysis parameters

In [ ]:
# Get the current directory
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
print(f"Check that this is root directory:{parent_dir}")

In [ ]:
##### Area-Specific Files ##### 

country_name = "Spain"

data_files = {
    "cellsites_file": 'ESP-1697916284-6wv8-cellsite.csv', # Cell sites data
    "mbbt_file": "MobileBB_Traffic_per_Subscr_per_Month.csv", # Mobile broadband traffic data
    "poi_file": "ESP-1697915895-xs2u-pointofinterest.csv",
    "poi_visibility_file": "ESP-1708423221-tgah-visibility.csv", # Visibility analysis output file
    "bwdistance_km_file_name": "bwdistance_km.csv", # Distance samples for channels with different bandwidth in km
    "bwdlachievbr_file_name": "bwdlachievbr_kbps.csv", # Achievable downlink bitrate for channels with different bandwidth in kbps.
    "pop_file": "population.tif", # Population density raster file name
    "area_file" : "area.gpkg", # Area contour file name
    "mbbsubscr_file" : "active-mobile-broadband-subscriptions_1711147050645.csv", # Mobile broadband subscriptions data
    "mbbtraffic_file" : "mobile-broadband-internet-traffic-within-the-country_1711147118571.csv", # Mobile broadband traffic data
    "mobile_coverage_file" : "mobile_coverage_merged_fixed_geom.gpkg" # Mobile coverage file, multipolygon data in geospatial format
}

##### Calculation Parameters #####

### Network Configuration ###

bw = 20 # Bandwidth, MHz
cco = 18 # Control Channel Overheads, %
fb_per_site = 3 # No. of Frequency Bands on Site
angles_num = 360 # PLACEHOLDER # Set the number of angles to be used for azimuth analysis  
rotation_angle = 60 # PLACEHOLDER # Define the rotation angle to create a sector +/-rotation_angle degrees clockwise and counter-clockwise  

### POI requirements ###

dlthtarg = 160 # Download throughput target in Mbps.

### Population information ###

oppopshare = 50 # % of Population on Operator
min_radius = 1000 # meters, minimum radius around cell site location for population calculation
max_radius = 3000  # meters, maximum radius should be divisible by 1000; maximum radius around cell site location for population calculation
radius_step = 1000 # meters, radius step size for population calculation

### Avg user traffic profile ###

nonbhu = 50 # Non-Busy Hour Usage, %

## Create an instance of the Capacity class

In [ ]:
# Create an instance of the Capacity class
mobilecapacity = Capacity(data_files, country_name,
                 bw, cco, fb_per_site, max_radius, min_radius, radius_step, angles_num,
                 rotation_angle, dlthtarg, nonbhu,rb_num_multiplier=5,nbhours=10, 
                 root_dir = parent_dir,enable_logging=False)

In [ ]:
# Check input data path
mobilecapacity.input_data_path

In [ ]:
# Check parent directory path
parent_dir

In [ ]:
print(f"The mobile broadband internet traffic per subscription per month in {country_name} is: {round(mobilecapacity.udatavmonth_pu,0)} GB per month according to {mobilecapacity.udatavmonth_year} data.")

## Available channel capacity

## Run capacity check for one POI

In [ ]:
# Test: check capacity for some random input values
sufficient_capacity_check = mobilecapacity.capacity_checker(d = 2000, popcd=1000, udatavmonth=mobilecapacity.udatavmonth_pu, pop=5000)[3]
print(f'Capacity is sufficient to connect the POI: {sufficient_capacity_check}.')

## Run buffer analysis for all POIs

In [ ]:
# Run buffer analysis
buffer_areas, poi_sufcapch_result = mobilecapacity.calculate_buffer_areas()

In [ ]:
# Buffer areas output data sample
buffer_areas.head(2)

In [ ]:
# POI capacity sufficiency output data sample
poi_sufcapch_result.head(2)

In [ ]:
# Number of True and False POI capacity sufficiency checks
poi_sufcapch_result.sufcapch.value_counts()

## Charts

In [ ]:
# Coordinate reference system:
crs = "4326"

# Chart data

# POI and cell sites area
area = gpd.read_file(f'{parent_dir}//data//input_data//{data_files["area_file"]}', crs=crs)
# Cell sites
cell_sites = pd.read_csv(f"{mobilecapacity.input_data_path}/{mobilecapacity.data_files['cellsites_file']}")
cell_sites = gpd.GeoDataFrame(cell_sites, geometry=gpd.points_from_xy(cell_sites.lon, cell_sites.lat), crs=crs)
# POIs
pois = pd.read_csv(f'{parent_dir}//data//input_data//{data_files["poi_file"]}')
pois = gpd.GeoDataFrame(pois, geometry=gpd.points_from_xy(pois.lon, pois.lat), crs=crs)
# Population density raster
raster = rasterio.open(f"{mobilecapacity.input_data_path}/{mobilecapacity.data_files['pop_file']}")
# Mobile coverage 
mc = gpd.read_file(f'{parent_dir}//data//input_data//{data_files["mobile_coverage_file"]}', crs=crs)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))  
  
area.plot(ax=ax, color = 'gray')
 
ax.set_title(f'Cell site and POI location area')  
ax.set_xlabel('Longitude')  
ax.set_ylabel('Latitude')  

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))  

area.plot(ax=ax, color = 'gray')

cell_sites.plot(ax=ax, color='yellow', markersize=5, label='Cell site locations') 

ax.set_title(f'Cell site locations (synthetic data for illustration purposes)')  
ax.set_xlabel('Longitude')  
ax.set_ylabel('Latitude')
ax.legend() 
 
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))  

area.plot(ax=ax, color = 'gray')

pois.plot(ax=ax, color = 'lightgreen', markersize=10, label="POI locations")  

ax.set_title(f'POI locations (synthetic data for illustration purposes)')  
ax.set_xlabel('Longitude')  
ax.set_ylabel('Latitude')
ax.legend() 
 
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))  

rio_show(raster, ax=ax)  
   
ax.set_title(f'Population density.')  
ax.set_xlabel('Longitude')  
ax.set_ylabel('Latitude')  

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))  

rio_show(raster, ax=ax)  
   
for radius in range(min_radius, max_radius+1, radius_step):
        buffer_areas[f'ring_{radius}'].plot(ax=ax, color='blue', edgecolor='k', alpha=0.5)
    
buffer_areas.plot(ax=ax, color='yellow', markersize=5, label = "Cell site locations") 
 
ax.set_title(f'Ring areas around cell sites for radiuses from {min_radius} m to {max_radius} m with {radius_step} m step.')  
ax.set_xlabel('Longitude')  
ax.set_ylabel('Latitude') 
ax.legend() 
 
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))  
  
rio_show(raster, ax=ax)  
  
for radius in range(min_radius, max_radius+1, radius_step):
        buffer_areas[f'clring_{radius}'].plot(ax=ax, color='blue', edgecolor='lightgrey', alpha=0.5)
  
buffer_areas.plot(ax=ax, color='yellow', markersize=5, label = "Cell site locations") 

ax.set_title('Mobile cellular service coverage areas')  
ax.set_xlabel('Longitude')  
ax.set_ylabel('Latitude')
ax.legend()
 
plt.show()  


In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))  

rio_show(raster, ax=ax)  
  
for radius in range(min_radius, max_radius+1, radius_step):
        buffer_areas[f'clring_{radius}'].plot(ax=ax, color='blue', edgecolor='lightgrey', alpha=0.5)
  
cell_sites.plot(ax=ax, color='yellow', markersize=5, label='Cell site locations')

pois = pois.merge(poi_sufcapch_result[['is_visible', 'sufcapch']], how='left', left_on='poi_id', right_index=True)

pois[pois['sufcapch'] == True].plot(ax=ax, color='lightgreen', markersize=10, label='POI: Sufficient Capacity')
pois[pois['sufcapch'] == False].plot(ax=ax, color='red', markersize=10, label='POI: Insufficient Capacity')

ax.set_title('Mobile cellular service capacity sufficiency check')
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
ax.legend()

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))  

area.plot(ax=ax, color = 'gray')
mc.plot(ax=ax, color = 'blue')

pois.plot(ax=ax, color = 'lightgreen', markersize=10, label="POI locations")
   
ax.set_title(f'POI locations (synthetic data for illustration purposes)')  
ax.set_xlabel('Longitude')  
ax.set_ylabel('Latitude')
ax.legend()  

plt.show()

## Mobile broadband traffic per mobile broadband subscription per month

In [ ]:
# Compile mobile traffic data based on the latest year ITU data available for both
# mobile-broadband Internet traffic (within the country) and active mobile-broadband subscriptions
# and calculates Mobile broadband internet traffic (within the country) per active mobile broadband subscription per month.
mobile_traffic_data = mobilecapacity.mbbtps()

In [ ]:
# Display mobile traffic data sample
mobile_traffic_data.head()

In [ ]:
# Group the data by 'entityIso_mbbsubscr' and calculate the mean traffic per subscriber per month
grouped_data = mobile_traffic_data.loc[mobile_traffic_data.loc[mobile_traffic_data.dataYear == 2022,:].index, ['entityIso_mbbsubscr','mbb_traffic_per_subscr_per_month']].set_index('entityIso_mbbsubscr')

# Define country 
start_position = 30 # the first country to be displayed on the chart (numeration starts with 0)
end_position = 50 # the last country to be displayed on the chart

# Sort the grouped data by the mean traffic values
sorted_data = grouped_data.sort_values(by = 'mbb_traffic_per_subscr_per_month', ascending=False)[start_position:end_position]

fig, ax = plt.subplots(figsize=(10, 10))

sorted_data.plot(kind='bar', ax=ax, color='skyblue', edgecolor='black')

ax.set_title('Mean Traffic per Subscriber per Month by Country in 2022')
ax.set_xlabel('Country ISO 3 code')
ax.set_ylabel('Mean Traffic per Subscriber per Month (GB)')
ax.grid(True)

plt.show()

In [ ]:
mobile_traffic_data.to_csv(f"{mobilecapacity.output_data_path}/MobileBB_Traffic_per_Subscr_per_Month.csv")